In [3]:
import psi4
import numpy as np
import sys
sys.path.append('../lib')
from P4toC4_aux import *

In [5]:
C4_cs = read_oldmos('../Cfour/SYM/OLDMOS.Cs', [6,1])

reading orbitals from ../Cfour/SYM/OLDMOS.Cs


In [6]:
psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", "sto-3g")
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("REFERENCE", "RHF")
psi4.core.set_global_option("D_CONVERGENCE", 1e-8)
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_output_file('output.dat', False)

# O
# H 1 0.96
# H 1 0.96 2 104.5

h2o = psi4.geometry("""
0 1
O           -0.005580556816     0.124770196240     0.000000000000
H           -1.409609476816    -1.024054213760     0.000000000000
H            1.498176963184    -0.956139343760     0.000000000000
units Bohr
no_reorient
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-74.96391080920293

In [7]:
mol=wf.molecule()
ptgr=mol.point_group()
print(ptgr.symbol(), ptgr.order())

cs 2


In [8]:
# Print a new basis set in GENBAS format
basisset=wf.basisset()
#print(basisset.genbas())

In [9]:
mapping, scale = basis_mapping(wf.basisset(), verbose=0)
mapping

array([0, 1, 4, 2, 3, 5, 6])

In [6]:
psi4.core.Vector.array_interface(wf.epsilon_a())

[array([-20.24262218,  -1.26148562,  -0.61196634,  -0.45026917,
          0.59134088,   0.72961407]),
 array([-0.39008602])]

In [11]:
# Psi4
Ca=wf.Ca()
#Cb=np.array(wf.Cb())

In [12]:
Ca.shape

((6, 6), (1, 1))

In [13]:
Ca.nph

(array([[ 9.94158616e-01,  2.33318779e-01,  3.25678536e-03,
          1.02160791e-01,  1.30288373e-01,  1.15041853e-02],
        [ 2.63047218e-02, -8.38213877e-01, -1.77263554e-02,
         -5.31043974e-01, -8.59997419e-01, -8.42061361e-02],
        [-8.57883491e-06,  1.50998145e-03, -6.04682049e-01,
          3.08286109e-02, -9.53036679e-02,  9.79720354e-01],
        [-4.21140198e-03,  1.24846086e-01, -2.86266450e-02,
         -7.74819295e-01,  7.33875191e-01,  9.11576117e-02],
        [-5.96714175e-03, -1.61730307e-01,  4.45858226e-01,
          2.75644066e-01,  7.29782613e-01,  8.93396408e-01],
        [-5.70344400e-03, -1.54033146e-01, -4.45125144e-01,
          2.89100795e-01,  8.41536318e-01, -7.56741617e-01]]),
 array([[1.]]))

In [16]:
#
#  at least for STO-3G the SO coefficients are identical, no reordering needed.
#
for i in range(wf.nirrep()):
    print(np.max(Ca.nph[i]-C4_cs[i]))

1.3528361320069848e-08
2.220446049250313e-16


reading orbitals from ../Cfour/SYM/OLDMOS.Cs


TypeError: 'builtin_function_or_method' object is not subscriptable

In [24]:
Ca_C4 = psi4_to_c4(Ca, mapping, scale)
Cb_C4 = psi4_to_c4(Cb, mapping, scale)

In [25]:
write_oldmos('PSIMOS', Ca_C4, Cbs=Cb_C4)